# Анализ базы данных сервиса для чтения книг по подписке

## Оглавление

1. [Введение](#intro)
2. [Исследование таблиц](#tables)
    1. [Подготовка окружения](#env)
    2. [Загрузка таблиц](#load)
    3. [Анализ дубликатов](#duplicates)
3. [Получение данных для анализа](#requests)
    1. [Анализ книг 21 века](#xxi)
    2. [Анализ обзоров и средних оценок](#review)
    3. [Анализ самого активного издательства](#publishers)
    4. [Анализ самого высокооцененного автора](#author)
    5. [Анализ активных пользователей](#users)
4. [Общий вывод](#total)

## Введение <a id="intro"></a>

### Задача

Нужно проанализировать базу данных крупного сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

Структура таблицы `books` (содержит данные о книгах):
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Структура таблицы `authors` (содержит данные об авторах):
- `author_id` — идентификатор автора;
- `author` — имя автора.

Структура таблицы `publishers` (содержит данные об издательствах):
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Структура таблицы `ratings` (содержит данные о пользовательских оценках книг):
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Структура таблицы `reviews` (содержит данные о пользовательских обзорах на книги):
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Исследование таблиц <a id="tables"></a>

### Подготовка окружения <a id="env"></a>

Загрузим библиотеки, необходимые для работы.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json

Настроим конфигурации для подключения к базе данных.

In [2]:
# получаем конфиг настроек БД из файла
with open('db_config.json', 'r') as file:
    db_config = json.load(file)

# коннектимся к БД
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Загрузка таблиц <a id="load"></a>

Загрузим все таблицы, а также выведем первые 5 строк из каждой.

In [3]:
def get_and_print_table(table_name, engine):
    '''
    функция для получения таблиц и вывода первых 5 строк
    принимает: название таблицы, коннектор к БД
    выводит: первые 5 строк таблицы
    возвращает: таблицу целиком
    '''
    # формируем запрос, запрашиваем всю таблицу, чтобы выгрузить ее в переменную для работы
    query = '''
    SELECT *
    FROM {};
    '''.format(table_name)
    
    # получаем таблицу в переменную
    table = pd.io.sql.read_sql(query, con=engine)
    
    # выводим название таблицы, и первые 5 строк
    print(table_name)
    display(table.head())
    display(table.info())
    
    return table
    
# выносим список названий таблиц в отдельную переменную
table_list = ['books', 'authors', 'publishers', 'ratings', 'reviews']

# создаем словарь для таблиц
tables = {}

# заполняем словарь таблицами
for table_name in table_list:
    tables[table_name] = get_and_print_table(table_name, engine)

books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

authors


...

None

reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

Вывод:
- В таблицах имеются данные о 1000 книг, 636 авторов, 340 издательствах, 6456 оценках и 2793 отзывах.
- Пропусков в данных нет.

### Анализ дубликатов <a id="duplicates"></a>

Проверим, есть ли явные дубликаты данных в каждой из таблиц.

In [4]:
for table_name in table_list:
    print('Количество дубликатов в таблице {}: {}'.format(table_name, tables[table_name].duplicated().sum()))

Количество дубликатов в таблице books: 0
Количество дубликатов в таблице authors: 0
Количество дубликатов в таблице publishers: 0
Количество дубликатов в таблице ratings: 0
Количество дубликатов в таблице reviews: 0


Явных дубликатов ни в одной из таблиц нет.

Также проверим, есть ли дублирующиеся оценки и отзывы от пользователей, то есть посмотрим наличие пользователей, оценивших или написавших отзыв по одной и той же книге несколько раз.

In [5]:
print('Количество дубликатов значений книги и пользователя в таблице ratings: {}'.format(
    tables['ratings'].loc[:, ['book_id', 'username']].duplicated().sum()
))
print('Количество дубликатов значений книги и пользователя в таблице reviews: {}'.format(
    tables['reviews'].loc[:, ['book_id', 'username']].duplicated().sum()
))

Количество дубликатов значений книги и пользователя в таблице ratings: 0
Количество дубликатов значений книги и пользователя в таблице reviews: 0


Вывод:
- Явных дубликатов ни в одной из таблиц нет.
- Также нет пользователей, которые оставили несколько оценок или отзывов по одной и той же книге.

## Получение данных для анализа <a id="requests"></a>

### Анализ книг 21 века <a id="xxi"></a>

Посчитаем, сколько книг вышло после 1 января 2000 года (не включительно).

In [6]:
# формируем запрос
query = '''
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01';
'''
    
# выводим результат
display(pd.io.sql.read_sql(query, con=engine))

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Анализ обзоров и средних оценок <a id="review"></a>

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [7]:
# формируем запрос
# используем LEFT JOIN, чтобы учитывать книги без обзоров и рейтингов, если такие есть
# сортируем по убыванию количества обзоров и рейтинга
query = '''
SELECT b.title,
    COUNT(DISTINCT re.review_id) AS reviews_count,
    AVG(ra.rating) AS avg_rating
FROM books AS b
LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
LEFT JOIN reviews AS re ON b.book_id = re.book_id
GROUP BY b.book_id
ORDER BY reviews_count DESC,
    avg_rating DESC;
'''

    
# выводим результат
display(pd.io.sql.read_sql(query, con=engine))

,title,reviews_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Book Thief,6,4.264151
4,The Glass Castle,6,4.206897
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,Essential Tales and Poems,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Выводы:
- В данных есть одна книга `Twilight (Twilight #1)` с самым большим количеством обзоров (7).
- Остальные книги имеют 6 обзоров и меньше.
- В данных присутствуют книги без единого обзора, но с проставленными рейтингами.

### Анализ самого активного издательства <a id="publishers"></a>

Найдем издательство, которое выпустило наибольшее число книг толще 50 страниц - так мы исключим из анализа брошюры. Выведем топ-5 издательств по количеству таких выпущенных книг.

In [8]:
# формируем запрос
# выводим топ-5 на случай, если будет несколько издательств с одинаковым количеством выпущенных книг
query = '''
SELECT pu.publisher,
    COUNT(DISTINCT b.book_id) AS count
FROM books AS b
JOIN publishers AS pu ON b.publisher_id = pu.publisher_id
WHERE b.num_pages > 50
GROUP BY pu.publisher_id
ORDER BY count DESC
LIMIT 5;
'''
    
# выводим результат
display(pd.io.sql.read_sql(query, con=engine))

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


Больше всего книг толще 50 страниц выпустило издательство `Penguin Books`. Они выпустили 42 книги.

### Анализ самого высокооцененного автора <a id="author"></a>

Найдем автора с самой высокой средней оценкой книг. Для анализа будем учитывать только книги с 50 и более оценками. Выведем топ-5 таких авторов.

In [9]:
# формируем запрос
# выводим топ-5 на случай, если будет несколько авторов с одинаковой оценкой книг
# для удобства список книг и их средних рейтингов вынесли в отдельную таблицу
query = '''
WITH avg_ratings AS
(
SELECT b.book_id,
    AVG(ra.rating) AS avg_rating
FROM books AS b
JOIN ratings AS ra ON b.book_id = ra.book_id
GROUP BY b.book_id
HAVING COUNT(DISTINCT ra.rating_id) >= 50
)

SELECT a.author,
    AVG(avg.avg_rating) AS avg_rating
FROM authors AS a
JOIN books AS b ON a.author_id = b.author_id
JOIN avg_ratings AS avg ON b.book_id = avg.book_id
GROUP BY a.author_id
ORDER BY avg_rating DESC
LIMIT 5;
'''
    
# выводим результат
display(pd.io.sql.read_sql(query, con=engine))

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


У автора `J.K. Rowling/Mary GrandPré` самая высокая средняя оценка книг (4.28), которые имеют 50 и более оценок.

### Анализ активных пользователей <a id="users"></a>

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [10]:
# формируем запрос
query = '''
WITH c AS
(
SELECT ra.username,
    COUNT(DISTINCT re.review_id) AS count_reviews
FROM ratings AS ra
LEFT JOIN reviews AS re ON ra.username = re.username
GROUP BY ra.username
HAVING COUNT(DISTINCT ra.rating_id) > 50
)

SELECT AVG(count_reviews)
FROM c;
'''
    
# выводим результат
display(pd.io.sql.read_sql(query, con=engine))

,avg
0,24.333333


Пользователи, которые поставили больше 50 оценок, в среднем пишут 24.3 обзора.

## Общий вывод <a id="total"></a>

- В таблицах имеются данные о 1000 книг, 636 авторов, 340 издательствах, 6456 оценках и 2793 отзывах.
- Пропусков и дубликатов в данных нет.
- После 1 января 2000 года вышло 819 книг.
- В данных есть одна книга `Twilight (Twilight #1)` с самым большим количеством обзоров (7 шт).
- Остальные книги имеют 6 обзоров и меньше.
- Также присутствуют книги без единого обзора, но с проставленными рейтингами.
- Больше всего книг толще 50 страниц выпустило издательство `Penguin Books` (42 книги).
- У автора `J.K. Rowling/Mary GrandPré` самая высокая средняя оценка книг (4.28), которые имеют 50 и более оценок.
- Пользователи, которые поставили больше 50 оценок, в среднем пишут 24.3 обзора.